In [ ]:
!unzip drive/MyDrive/archive.zip

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [8]:
batch_size = 50
img_height = 100
img_width = 100

train_data_dir = "fruits-360/Training"
test_data_dir = "fruits-360/Test"

In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_dir,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 67692 files belonging to 131 classes.


In [10]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_data_dir,
  validation_split=0.2,
  subset="validation",
  image_size=(img_height, img_width),
  seed=42,
  batch_size=batch_size)

Found 22688 files belonging to 131 classes.
Using 4537 files for validation.


In [11]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_data_dir,
  validation_split=0.2,
  subset="training",
  image_size=(img_height, img_width),
  seed=42,
  batch_size=batch_size)

Found 22688 files belonging to 131 classes.
Using 18151 files for training.


In [12]:
"""
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
"""

'\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(10, 10))\nfor images, labels in train_ds.take(1):\n  for i in range(9):\n    ax = plt.subplot(3, 3, i + 1)\n    plt.imshow(images[i].numpy().astype("uint8"))\n    plt.title(class_names[labels[i]])\n    plt.axis("off")\n'

In [13]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(50, 100, 100, 3)
(50,)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [15]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [16]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.0 1.0


In [17]:
one_hot_encoded_ds = normalized_ds.map(lambda x, y: (x, tf.one_hot(y, 131)))
image_batch, labels_batch = next(iter(one_hot_encoded_ds))
first_label = labels_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(first_label)
print(np.min(first_image), np.max(first_image)) 

tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(131,), dtype=float32)
0.0 1.0


In [18]:
one_hot_train_ds = one_hot_encoded_ds
one_hot_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), tf.one_hot(y, 131)))
one_hot_test_ds = test_ds.map(lambda x, y: (normalization_layer(x), tf.one_hot(y, 131)))

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 2,input_shape=(100,100,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 32,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 64,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 128,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(131,activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 16)      208       
_________________________________________________________________
activation (Activation)      (None, 100, 100, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0

In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print('Compiled!')

Compiled!


In [21]:
epochs=5

model.fit(
  one_hot_train_ds,
  validation_data=one_hot_val_ds,
  epochs=epochs
)

Epoch 1/5
1354/1354 [==============================] - 124s 46ms/step - loss: 2.4088 - accuracy: 0.4156 - val_loss: 0.2588 - val_accuracy: 0.9253
Epoch 2/5
1354/1354 [==============================] - 33s 24ms/step - loss: 0.1950 - accuracy: 0.9343 - val_loss: 0.2382 - val_accuracy: 0.9403
Epoch 3/5
1354/1354 [==============================] - 32s 24ms/step - loss: 0.1009 - accuracy: 0.9666 - val_loss: 0.1586 - val_accuracy: 0.9641
Epoch 4/5
1354/1354 [==============================] - 32s 24ms/step - loss: 0.0698 - accuracy: 0.9779 - val_loss: 0.2403 - val_accuracy: 0.9522
Epoch 5/5
1354/1354 [==============================] - 32s 24ms/step - loss: 0.0566 - accuracy: 0.9825 - val_loss: 0.2191 - val_accuracy: 0.9641


In [22]:
test_loss, test_acc = model.evaluate(one_hot_test_ds, verbose=2)
print("loss :", test_loss)
print("accuracy :", test_acc)

364/364 - 21s - loss: 0.2451 - accuracy: 0.9639
loss : 0.24509862065315247
accuracy : 0.9639138579368591
